In [ ]:
%load_ext autoreload

In [ ]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd
import logging

In [ ]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions

In [ ]:
from pydrake.geometry.optimization_dev import (CspaceFreePolytope, 
                                               FindPolytopeGivenLagrangianOptions, 
                                               FindSeparationCertificateGivenPolytopeOptions, 
                                               BilinearAlternationOptions,
                                               BinarySearchOptions,
                                               SeparatingPlaneOrder,
                                               EllipsoidMarginCost)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

We first set up a simple 2-DOF system and visualize both the plant and the configuration constraint.Click on the two links at the bottom to view the plant and the configuration space.

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [ ]:
#construct our robot
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))
locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()
idx = 0
q0 = [0.0, 0.0]
val = 1.7
q_low  = [-val, -val, 0]
q_high = [val, val,  0]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_s = []
for q in [q_low, q_high]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star)[:2])

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat1.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

context = visualizer.diagram_context

## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the two degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [ ]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


In [ ]:
# # filter fused joints self collisions so they don't interfere with collision engine
# digaram = visualizer.diagram
# context = visualizer.diagram_context
# sg_context = scene_graph.GetMyContextFromRoot(context)
# inspector = scene_graph.model_inspector()

# pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

# gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
# get_name_of_gid = lambda gid : inspector.GetName(gid)
# gids.sort(key=get_name_of_gid)
# right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
# left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

# right_sweeper_fused_col_geom = right_sweeper_gids[2:]
# right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
# left_sweeper_fused_col_geom = left_sweeper_gids[4:]
# left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

# right_sweeper_end_gid = right_sweeper_gids[-1]
# left_sweeper_end_gid = left_sweeper_gids[-1]
# id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
#                        ]
# visualizer.collision_pairs_of_interest = id_pairs_of_interest


# Generate and Certify Regions

Around some nominal seed postures, we will grow certified regions by seeding our alternation algorithm using a small initial polytope.

In [ ]:
# Some seedpoints
seed_points_q = np.array([   [0.0, 0],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.4,-1.3]
                              ])
seed_points = np.array([Ratfk.ComputeSValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)
    


default_scale = 1e-2
L1_ball = HPolyhedron.MakeL1Ball(2)
Linf_ball = HPolyhedron.MakeBox(-np.ones(2), np.ones(2))

template_C = np.vstack([L1_ball.A(), Linf_ball.A()])
template_d = np.hstack([default_scale*L1_ball.b(), default_scale/np.sqrt(2)*Linf_ball.b()])


def make_default_polytope_at_point(seed_point):
    return HPolyhedron(template_C, template_d + template_C @ seed_point)


# colors to plot the region. Chosen for color-blind compatibility
colors_dict = {
    0: (144,144,144),
    1:(30,136,229), # bluish
    2: (255, 193, 7), # gold
    3: (0, 140, 6), # green    
}

initial_regions = [make_default_polytope_at_point(s) for s in seed_points]
if do_viz:
    visualizer.plot_regions(initial_regions,
                            region_suffix='_default_regions',
                            colors = list(colors_dict.values()),
                            wireframe = False,
                           opacity = 0.2)

In [ ]:
# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 1)


find_polytope_given_lagrangian_option = FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = False

find_separation_certificate_given_polytope_options = FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.num_threads = -1
find_separation_certificate_given_polytope_options.verbose = False
find_separation_certificate_given_polytope_options.solver_options = solver_options
find_separation_certificate_given_polytope_options.ignore_redundant_C = True

bilinear_alternation_options = BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 20
bilinear_alternation_options.convergence_tol = 1e-5
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option
bilinear_alternation_options.find_lagrangian_options = find_separation_certificate_given_polytope_options

binary_search_options = BinarySearchOptions()
binary_search_options.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options.scale_min = 1
binary_search_options.scale_max = 100
binary_search_options.max_iter = 50


cspace_free_polytope = CspaceFreePolytope(plant, scene_graph, SeparatingPlaneOrder.kAffine, q_star)

# Growing regions with bilinear alternations search

### As the initial regions are fairly small, they won't contain collisions and so can be directly fed into the bilinear alternation algorithm.

In [ ]:
# now we grow certified regions around each seedpoint
bilinear_alternation_region_certificates = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    bilinear_alternation_options.find_polytope_options.s_inner_pts = s
    bilinear_alternation_region_certificates[tuple(s)] = cspace_free_polytope.SearchWithBilinearAlternation(set(), 
                                                                              initial_region.A(),
                                                                              initial_region.b(),
                                                                              bilinear_alternation_options)
bilinear_alternation_certified_regions = {s : [HPolyhedron(result.C, result.d) for result in search_result_list if result is not None] 
                     for s, search_result_list in bilinear_alternation_region_certificates.items() if search_result_list is not None}

In [ ]:
if do_viz:
    for i, s in enumerate(seed_points):
        visualizer.plot_regions(bilinear_alternation_certified_regions[tuple(s)],
                                region_suffix=f"_{i}_BIL_ALT",
                                wireframe = False,
                                opacity = 0.2)

# Growing regions with binary search

### While the bilinear alternation scheme has the flexibility to search for fairly flexible polytopes, it can be relatively slow. We can search for larger regions faster by uniformly growing our polytopes using binary search

In [ ]:
# now we grow certified regions around each seedpoint
binary_search_region_certificates = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    binary_search_region_certificates[tuple(s)] = cspace_free_polytope.BinarySearch(set(),
                                                                                    initial_region.A(),
                                                                                    initial_region.b(),
                                                                                    s,
                                                                                    binary_search_options)
binary_search_certified_regions = {s : [HPolyhedron(result.C, result.d)] 
                     for s, result in binary_search_region_certificates.items() if result is not None}

In [ ]:
if do_viz:
    for i, s in enumerate(seed_points):
        visualizer.plot_regions(binary_search_certified_regions[tuple(s)],
                                region_suffix=f"_{i}_BIN_SEARCH",
                                wireframe = False,
                                opacity = 0.2)

## Combining Binary Search and Bilinear Alternation
### Of course, we can get the best of both worlds by combining the two methods

In [ ]:
# now we grow certified regions around each seedpoint
binary_and_bilinear_certified_regions = binary_search_certified_regions.copy()
for i, (s, region_start) in enumerate(binary_and_bilinear_certified_regions.items()):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    bilinear_alternation_options.find_polytope_options.s_inner_pts = np.array(s)
    certificates = cspace_free_polytope.SearchWithBilinearAlternation(set(),
                                                                      region_start[0].A(),
                                                                      region_start[0].b(), 
                                                                      bilinear_alternation_options)
    binary_and_bilinear_certified_regions[s] += [HPolyhedron(result.C, result.d) for result in certificates if result is not None]

In [ ]:
if do_viz:
    for i, s in enumerate(seed_points):
        visualizer.plot_regions(binary_and_bilinear_certified_regions[tuple(s)],
                                region_suffix=f"_{i}_BIN_AND_BIL",
                                wireframe = False,
                                opacity = 0.2)

## Seeding and Certifying with a Stronger Heuristic
We have also implemented another, strong heuristic for proposing good initial regions based on non-linear optimization. See Appendix TODO of our paper TODO for details

In [ ]:
iris_regions = []
iris_ellipses = []

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.configuration_space_margin = 1e-3
iris_options.relative_termination_threshold = 0.001

def promote_region_to_3d(region, width = 0.2):
    A = block_diag(region.A(), np.array([-1,1])[:, np.newaxis])
    b = np.append(region.b(), width*np.ones(2))

    return HPolyhedron(A,b)
    
for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, np.zeros((2,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q)
    r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), q_star, iris_options)
    iris_regions.append(r)
    iris_ellipses.append(r.MaximumVolumeInscribedEllipsoid)
    

if do_viz:
    visualizer.plot_regions(iris_regions,
                            region_suffix='_iris_regions',
                            colors = list(colors_dict.values()),
                            wireframe = False,
                           opacity = 0.2)

### These regions tend to be very large, but typically are not completely collision free. We can use the binary search method to find a uniform shrinking of these regions to prove their safety and then again improve them with bilinear alternations.

In [ ]:
binary_search_options_for_iris = BinarySearchOptions()
binary_search_options_for_iris.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options_for_iris.max_iter = 50

In [ ]:
binary_search_region_certificates_for_iris = dict.fromkeys([tuple(s) for s in seed_points])
for i, (s, initial_region) in enumerate(zip(seed_points, iris_regions)):
    print(f"starting seedpoint {i+1}/{len(iris_regions)}")
    certificate = cspace_free_polytope.BinarySearch(set(),
                                                    initial_region.A(),
                                                    initial_region.b(), 
                                                    initial_region.MaximumVolumeInscribedEllipsoid().center(), 
                                                    binary_search_options_for_iris)
    binary_search_region_certificates_for_iris[tuple(s)] = [HPolyhedron(certificate.C, certificate.d)]

In [ ]:
if do_viz:
    for i, s in enumerate(seed_points):
        visualizer.plot_regions(binary_search_region_certificates_for_iris[tuple(s)],
                                region_suffix=f"_{i}_BIN_SEARCH_IRIS_REGIONS",
                                wireframe = False,
                                opacity = 0.2)

In [ ]:
# finally we grow the regions again using bilinear alternations.
bilinear_certified_regions_iris = binary_search_region_certificates_for_iris.copy()
for i, (s, region_start) in enumerate(bilinear_certified_regions_iris.items()):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    bilinear_alternation_options.find_polytope_options.s_inner_pts = np.array(s)
    certificates = cspace_free_polytope.SearchWithBilinearAlternation(set(),
                                                                      region_start[0].A(),
                                                                      region_start[0].b(), 
                                                                      bilinear_alternation_options)
    bilinear_certified_regions_iris[s] += [HPolyhedron(result.C, result.d) for result in certificates if result is not None]

In [ ]:
if do_viz:
    for i, s in enumerate(seed_points):
        visualizer.plot_regions(bilinear_certified_regions_iris[tuple(s)],
                                region_suffix=f"_{i}_BIL_IRIS_REGIONS",
                                wireframe = False,
                                opacity = 0.2)